<a href="https://colab.research.google.com/github/eshnil2000/google-colab/blob/master/linear_regression_home_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Parameters
learning_rate = 0.6
training_epochs = 25
display_step = 1
batch_size=64

In [20]:
!git clone https://github.com/eshnil2000/google-colab.git
%cd google-colab
#Install required modules
!pip install tensorflow numpy matplotlib pandas sklearn
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from tqdm import tqdm

Cloning into 'google-colab'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 259 (delta 53), reused 27 (delta 9), pack-reused 147
Receiving objects: 100% (259/259), 30.32 MiB | 32.48 MiB/s, done.
Resolving deltas: 100% (118/118), done.
/content/google-colab/google-colab/google-colab


In [0]:
import pandas as pd
df = pd.read_csv('data/housepricedata.csv')
dataset = df.values
train_X = dataset[:,0:10]
train_Y = dataset[:,10]
n_batches = int(train_X.shape[0]/batch_size)

In [0]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(train_X)
train_X=X_scale

In [0]:
depth=len(np.unique(train_Y))
indices=train_Y
onehot_Y=tf.one_hot(indices, depth)
with tf.Session() as sess:
    onehot_Y=sess.run(onehot_Y)
sess.close()

In [50]:
# Run optimization and test
loss_history = []
acc_history = []
optim_history=[]

X = tf.placeholder(tf.float32,shape=(batch_size,10), name="input")
Y = tf.placeholder(tf.float32,shape=(batch_size,2), name="label")
# Set model weights
# Create weights and bias
w = tf.Variable(tf.random_normal(shape=(10,2), stddev=0.1), name="weights")
b = tf.Variable(tf.zeros(shape=(1,2)), name='bias')

# calculate scores
logits = tf.matmul(X, w)+b

# Entropy cost function and loss
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y)
loss = tf.reduce_mean(entropy)

# Define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

iterations = len(train_Y*training_epochs)

dataset = tf.data.Dataset.from_tensor_slices((train_X, onehot_Y))
dataset = dataset.repeat(training_epochs).batch(batch_size)
iterator = dataset.make_one_shot_iterator()

data_X, data_Y = iterator.get_next()
data_Y = tf.cast(data_Y, tf.int32)

c1=[]

with tf.Session() as sess, tqdm(total = iterations) as pbar:
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    for i in range(n_batches-1):
      try:
        while True:

          _, loss_value = sess.run([optimizer, loss], feed_dict={X: data_X.eval(), Y: data_Y.eval()})

          loss_history.append(loss_value)
          pbar.update(batch_size)
          loss_history.append(loss_value)
          optim_history.append(optim_value)
      except tf.errors.OutOfRangeError:
        pass

18240it [00:02, 8382.37it/s]


In [51]:
dataset = tf.data.Dataset.from_tensor_slices((train_X, onehot_Y))
dataset = dataset.repeat(training_epochs).batch(batch_size)
iterator = dataset.make_one_shot_iterator()

data_X, data_Y = iterator.get_next()
data_Y = tf.cast(data_Y, tf.int32)

# Prediction
with tf.Session() as sess, tqdm(total = len(onehot_Y)) as pbar:
  sess.run(tf.global_variables_initializer())
  total_correct_preds = 0
  for i in range(n_batches):
    logits_batch = sess.run(logits, feed_dict={X: data_X.eval(), Y: data_Y.eval()})
    preds = tf.nn.softmax(logits_batch)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(data_Y.eval(), 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
    total_correct_preds += sess.run(accuracy)
    pbar.update(batch_size)

print("Test accuracy is {0}".format(total_correct_preds/train_X.shape[0]))

 96%|█████████▋| 1408/1460 [00:14<00:00, 103.27it/s]

Test accuracy is 0.47534246575342465
